In [1]:
import re
from pdfminer.high_level import extract_text

def get_atvs(directory, filename, separator):

    # Carregando o texto do arquivo PDF utf 8
    texto = extract_text(f'{directory}/{filename}', codec='utf-8')

    # Expressão regular para capturar o texto entre "Atividades:" e o padrão especificado "LETRA,LETRA,LETRA,NUMERO,NUMERO,PONTO,NUMERO,NUMERO"
    padrao = re.compile(r'Atividades:(.*?)(?=\b[A-Z]{3}\d{2}\.\d{2}\b|$)', re.DOTALL)

    # Encontrar todas as correspondências
    atividades = padrao.findall(texto)


    if separator == 'paragraph':

        return [clean_atvs(text) for text in atividades]

    elif separator == 'phrase':

        # Processar cada conjunto de atividades para separá-las corretamente
        atividades_separadas = [re.split(r'\n\d+\.\s*', ativ.strip()) for ativ in atividades]

        # Achatando a lista de listas em uma única lista e filtrando entradas vazias
        atividades_finais = [item for sublist in atividades_separadas for item in sublist if item]

        return [clean_atvs(text) for text in atividades_finais]
        

In [2]:
import unidecode
import re

def clean_atvs(texto):
    # Normaliza o texto removendo acentos e convertendo para minúsculas
    texto_limpo = unidecode.unidecode(texto).lower()

    # Remove números seguidos por pontos, e caracteres não-alfabéticos exceto espaços
    texto_limpo = re.sub(r'\d+\.', '', texto_limpo)
    texto_limpo = re.sub(r'[^a-z\s]', '', texto_limpo)

    # Remove múltiplos espaços e a palavra 'atividade'
    texto_limpo = re.sub(r'\s+', ' ', texto_limpo)
    texto_limpo = re.sub(r'\batividade\b', '', texto_limpo)

    # Remove espaços extras no início e no final
    texto_limpo = texto_limpo.strip()
    

    return None if texto_limpo == '' else texto_limpo


In [3]:
import os
directory_list = os.listdir('Corpus')

corpus = {}
for directory in directory_list:
    file_list = os.listdir(f'Corpus/{directory}')
    for file in file_list:
        atvs = get_atvs(f'Corpus/{directory}', file, separator='paragraph')
        atvs = [atv for atv in atvs if atv]
        corpus[directory] = corpus.get(directory, []) + atvs

#Save corpus to json
import json
with open('corpus.json', 'w', encoding='utf-8') as f:
    json.dump(corpus, f, ensure_ascii=False, indent=4)

#### TF IDF with SVM

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


atividades = []
labels = []

for label,atvs in corpus.items():
    if atvs != None:
        atividades.extend(atvs)
        labels.extend([label]*len(atvs))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(atividades, labels, test_size=0.2, random_state=10, shuffle=True)

vectorizer = TfidfVectorizer(stop_words=stop_words)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [7]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Treinando o modelo SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = svm_model.predict(X_test)

# Avaliando o modelo
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7857142857142857


### Word2Vec

In [8]:
import gensim